# MORD : Multi-class Classifier for Ordinal Regression

In [117]:
import pandas as pd
import numpy as np
import mord as m
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import backend as K
from keras.layers import Lambda
sess = tf.InteractiveSession()

C:\Users\giaco\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [168]:
train =  pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
vali = pd.read_csv("vali.csv")
df = pd.concat((train,test,vali))
df =df[["cosine_y","query","numWordsQuery","numWordsCV","numCommonWords","score"]]

In [63]:
X1 = df[["cosine_y","query","numWordsQuery","numWordsCV","numCommonWords"]]
y1 = df["score"]

In [120]:
def dcg(score):
    res=[]
    for i in range((int(score.shape[0]))):
        #print(i)
        #print(score[i])
        rel_i = (score)[i]
        #print(rel_i)
        log = K.log(sess.run(tf.to_float(i+2)))
        #print(sess.run(log))
        #frac = K.tf.divide(rel_i,log)
        divResult = Lambda(lambda x: x[0]/x[1])([rel_i,log])

        res.append(sess.run(divResult))
    return np.sum(res)
def idcg(score_true):
    frac=[]
    y_sorted = sorted(sess.run(score_true),reverse=True)
    #print((y_sorted))
    for i in range(len(y_sorted)):
        num = 2**(y_sorted[i])-1
        #print(num)
        denom = np.log(i+2)
        #print (denom)
        frac.append(num/denom)
        #print(frac)
    return np.sum(frac)
def nDCG(score,score_true):
    return dcg(score)/idcg(score_true)


In [228]:
grouped = df.groupby(by='query')
mord1 = m.LogisticAT()
ndcgMord=[]

for i,j in grouped:
    if len(j)>=10:
        x=j.drop(["query","score"],axis=1)
        X = x.values
        y = j.score.values#.reshape(-1, 1)
        #print(y)
        X_train, X_test, y_train, y_test = train_test_split(X,y  , test_size=0.33, random_state=42)
        print("lunghezza train set",len(X_train))
        print("score train:",y_train)
        mord1.fit(X_train,y_train)
        nu=0
        pred=[]
        true =y_test
        for row in X_test:
            #print("input x:",row)
            pred.append(mord1.predict(row))
            #print("predict y:",pred)
            #print("true y:", y_test[nu])
            nu=nu+1
        print(pred)
        print(true)
        vec1 = tf.constant(np.array(pred),dtype=tf.float32)
        vec2 = tf.constant(np.array(true),dtype=tf.float32)
        print("metric nDCG:",nDCG(vec1,vec2))
        ndcgMord.append(nDCG(vec1,vec2))
        
        
    #else:
        #print(i,"not enough rows")

lunghezza train set 7
score train: [4 2 4 3 2 4 3]
[array([2]), array([3]), array([3]), array([3])]
[3 4 4 4]
metric nDCG: 0.19111123612846848
lunghezza train set 7
score train: [3 2 2 3 4 3 4]
[array([3]), array([3]), array([3]), array([3])]
[2 2 2 5]
metric nDCG: 0.21535261109459003
lunghezza train set 10
score train: [2 2 2 4 3 3 2 3 2 4]
[array([3]), array([3]), array([3]), array([2]), array([2]), array([2])]
[4 4 3 2 2 3]
metric nDCG: 0.2631985886695165
lunghezza train set 6
score train: [4 4 3 3 4 5]
[array([4]), array([4]), array([4]), array([4])]
[3 3 4 3]
metric nDCG: 0.39513820369402003
lunghezza train set 6
score train: [1 5 5 3 1 4]


ValueError: Values in y must be [0 1 2 3], instead got [0 2 3 4]

In [229]:
np.mean(ndcgMord)

0.26620015989664875

## Random forest

In [174]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler


In [178]:
X = df[["cosine_y","query","numWordsQuery","numWordsCV","numCommonWords"]]
y = df["score"]
X_train, X_test, y_train, y_test = train_test_split(X,y  , test_size=0.33, random_state=42)
# Feature Scaling
scaler = StandardScaler()


In [191]:
clf = RandomForestClassifier(n_estimators=100, max_depth=50,criterion = 'entropy', random_state=0)
clf.fit(X_train, y_train)
print("feature importance:",clf.feature_importances_)
predictions = (clf.predict(X_test))
test_labels = y_test

feature importance: [0.27628223 0.18950626 0.14643351 0.31838517 0.06939283]


In [192]:
# Performance metrics
errors = abs(predictions - test_labels)
print('Metrics for Random Forest Trained on Original Data')
print('Average absolute error:', round(np.mean(errors), 2), 'degrees.')
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Metrics for Random Forest Trained on Original Data
Average absolute error: 1.05 degrees.
Accuracy: 54.83 %.


In [226]:
mape

110     50.000000
79      66.666667
5       33.333333
77      20.000000
181     20.000000
284     50.000000
10      80.000000
100      0.000000
78      20.000000
350     60.000000
55     100.000000
118      0.000000
109     50.000000
29       0.000000
371      0.000000
234     50.000000
30      75.000000
212     66.666667
184      0.000000
86      25.000000
2      200.000000
1       40.000000
165      0.000000
37      25.000000
370     66.666667
318    100.000000
72      50.000000
135     40.000000
186     50.000000
70      33.333333
          ...    
250    100.000000
287    300.000000
158    200.000000
30      20.000000
167      0.000000
54     100.000000
124      0.000000
46      75.000000
93       0.000000
264      0.000000
108     33.333333
272      0.000000
40       0.000000
110      0.000000
66      25.000000
26     100.000000
9      100.000000
33      33.333333
107      0.000000
265      0.000000
2        0.000000
9       40.000000
69      80.000000
231     50.000000
113      0

In [200]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV

In [213]:
crossvalidation=KFold(n_splits=10,shuffle=True,random_state=1)
score=(cross_val_score(clf,X,y,scoring='accuracy', cv=crossvalidation,n_jobs=1))
print(score)

[0.39344262 0.29508197 0.36065574 0.29508197 0.41666667 0.51666667
 0.45       0.41666667 0.48333333 0.45      ]


In [203]:
GBC=GradientBoostingClassifier()
search_grid={'n_estimators':[100,500,1000],'learning_rate':[.001,0.01,.1],'max_depth':[1,3,5],'subsample':[.5,.75,1],'random_state':[1]}
search=GridSearchCV(estimator=GBC,param_grid=search_grid,scoring='accuracy',n_jobs=3,cv=crossvalidation)

In [204]:
search.fit(X,y)
search.best_params_

C:\Users\giaco\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'learning_rate': 0.01,
 'max_depth': 5,
 'n_estimators': 100,
 'random_state': 1,
 'subsample': 0.5}

In [211]:
ada2=GradientBoostingClassifier(n_estimators=60,learning_rate=0.01,subsample=0.5,max_depth=5,random_state=1)
score=np.mean(cross_val_score(ada2,X,y,scoring='accuracy',cv=crossvalidation,n_jobs=1))
score

0.4225136612021858

# SVM

In [145]:
# gaussian kernel
from sklearn.svm import SVC

In [151]:
X = df[["cosine_y","query","numWordsQuery","numWordsCV","numCommonWords"]]
y = df["score"]
X_train, X_test, y_train, y_test = train_test_split(X,y  , test_size=0.33, random_state=42)


In [156]:
svclassifier =  SVC(kernel='rbf',C=1,gamma=0.1)
svclassifier.fit(X_train, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [157]:
#prediction
y_pred = svclassifier.predict(X_test)

In [214]:
from sklearn.metrics import classification_report, confusion_matrix
mat = (confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.29      0.83      0.43        59
           2       0.23      0.09      0.13        34
           3       0.00      0.00      0.00        35
           4       0.23      0.09      0.13        33
           5       0.00      0.00      0.00        39

   micro avg       0.28      0.28      0.28       200
   macro avg       0.15      0.20      0.14       200
weighted avg       0.16      0.28      0.17       200



In [222]:
precision=np.diag(mat) / mat.sum(axis=1)

In [225]:
(precision)

array([0.83050847, 0.08823529, 0.        , 0.09090909, 0.        ])

In [221]:
mat.sum(axis=1)

array([59, 34, 35, 33, 39], dtype=int64)